In [ ]:
import json
import urllib.request

In [ ]:
def make_token():
    token = '&access_token='
    file_name = input('Название файла, в котором содержится _только_ токен: ')
    if not file_name:
        token = ""
    else:
        try:
            with open(file_name, 'r', encoding='utf-8') as f:
                tok = f.read().strip()
                token += tok
        except FileNotFoundError:
            token = ''
    return token

In [ ]:
def add_users():
    users = str(input("Введите никнэймы гитхаба через запятую БЕЗ ПРОБЕЛА: "))
    userlist = users.split(',')
    return userlist

In [ ]:
def choose_user(userlist):
    user = 'костыль'
    while user not in userlist:
        user = input('Выберите пользователя и введите его ник: ')
    return str(user)

In [ ]:
def download_info(user, token, part='repos'):
    info = []
    page = 0
    while len(info) == 100 * page:
        page += 1
        url = 'https://api.github.com'
        url += '/users/%s/%s?page=%s&per_page=100%s' % (user, part, page, token)
        j = urllib.request.urlopen(url)
        text = j.read().decode('utf-8')
        page_info = json.loads(text)
        info.extend(page_info)
    return info

In [ ]:
def u_repositories(user, info):
    start = 'Список репозиториев пользователя %s: ' % (user)
    print(start)
    for rep in info:
        name = rep['name']
        if rep['description']:
            descr = rep['description']
        else:
            descr = '---'
        output = '%s:\t%s' % (name, descr)
        print(output)

In [ ]:
def u_languages(user, info):
    lang_dict = {}
    for rep in info:
        if rep['language']:
            if rep['language'] in lang_dict:
                lang_dict[rep['language']] += 1
            else:
                lang_dict[rep['language']] = 1
    output = 'Языки, на которых пишет пользователь %s: ' % user
    print(output)
    for lang in sorted(lang_dict, key=lang_dict.get, reverse=True):
        amount = str(lang_dict[lang])
        output = '%s\t(%s)' % (lang, amount)
        print(output)

In [ ]:
def who_has_more(userlist, token):
    all_users = {}
    for user in userlist:
        all_users[user] = len(download_info(user, token))
    range_list = sorted(all_users, key=all_users.get, reverse=True)
    top = range_list[0]
    users = ','.join(userlist)
    output = "Из [%s] больше всего репозиториев у %s" % (users, top)
    print(output)

In [ ]:
def final_output(dict_name, descr):
    rating = sorted(dict_name, key=dict_name.get, reverse=True)
    top = rating[0]
    output = 'Самый популярный %s: %s!' % (descr, top)
    print(output)

In [ ]:
def popular_lang(userlist, token):
    langs = {}
    for user in userlist:
        info = download_info(user, token)
        for rep in info:
            if rep['language']:
                language = rep['language']
                if language in langs:
                    langs[language] += 1
                else:
                    langs[language] = 1
    descr = 'язык'
    final_output(langs, descr)

In [ ]:
def followers(userlist, token):
    folls = {}
    part = 'followers'
    for user in userlist:
        u_followers = download_info(user, token, part)
        folls[user] = len(u_followers)
    descr = 'блогер'
    final_output(folls, descr)

In [ ]:
def print_userinfo(user, info):
    u_repositories(user, info)
    print('\n')
    u_languages(user, info)

In [ ]:
def print_comparisons(userlist, token):
    who_has_more(userlist, token)
    popular_lang(userlist, token)
    followers(userlist, token)

In [ ]:
def main():
    token = make_token()
    userlist = add_users()
    user = choose_user(userlist)
    info = download_info(user, token)
    print_userinfo(user, info)
    print_comparisons(userlist, token)